I Подготовка данных

In [ ]:
Parse Information accretion file
Input -> txt file (GO:0000003 3.27)
Output -> dict(ключи - GO, значения - веса)

Переменная: ia_dict
Метод: parsers/ia_parser

-------

Parse obo file
Input -> obo file (go-basic.obo)
Output -> dict(ключи - namespace (BPO, CCO, MFO), значения:
{'name': 'mitochondrion inheritance',
 'namespace': 'biological_process',
 'def': '"The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton." [GOC:mcc, PMID:10873824, PMID:11389764]',
 'alt_id': [],
 'rel': ['GO:0048308', 'GO:0048311']})

Переменная: term_dict
Метод: parser/obo_parser

-------

Формирование листа с онтологиями в виде графа
Input -> term_dict
Output -> list[Graph]

Переменная: ontologies
Метод: main/for ns, terms_dict in obo_parser(args.obo_file).items():

-------

Формирование списка с файлами предсказаний моделей

Input -> path to prediction folder
Output -> list[str]

Переменная: pred_files
Метод: main/for root, dirs, files in os.walk(pred_folder):

-------

Parse ground truth

За ground truth берем часть train_terms.tsv (test_terms.tsv у нас нет)

Input -> gt_file, ontologies
Output -> dict(namespace: объект GroundTruth(ids, matrix, namespace))

Переменная: gt
Метод: parser/gt_parser

-------

Формируем маccив с пороговыми значения от 0.001 до 0.999

Переменная: tau_arr
Метод: main/np.arange(args.th_step, 1, args.th_step)

II Формирование итогового отчета dfs -> pd.DataFrame

In [ ]:
Для каждого файла из pred_files собираем данные
Input -> файл с предсказаниями
Output -> лист с объектами Prediction

Переменная: prediction
Метод: parser/pred_parser

-------

Оцениваем предсказания по списку метрик

["cov", "pr", "rc", "f", "wcov", "wpr", "wrc", "wf", "mi", "ru", "s"]

Переменная: df_pred
Метод: evaluation/evaluate_prediction

На выходе получаем 1 файл csv evaluation_all.tsv a table containing the full evaluation, i.e. assessment measures for each threshold. This file is used as input to generate the plots (see below)

И отдельный файл для каждой метрики с лучшими строками evaluation_best_{}.tsv